In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
! export TF_CPP_MIN_LOG_LEVEL=2

In [3]:
# gpu check
! nvidia-smi

Sat Dec  2 13:12:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3A:00.0 Off |                    0 |
| N/A   31C    P0    42W / 300W |     15MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [4]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [5]:
# from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
# # from tensorflow.keras.models import Model
# from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
# from sklearn.utils.class_weight import compute_class_weight
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
# import tensorflow.keras.backend as K
# from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
# from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
# from google.colab import files
import gc

In [6]:
import tensorflow as tf
print(tf. __version__)

2.12.0


In [7]:
pickle.format_version

'4.0'

## Load Data

In [8]:
# path to glove data
data_path = 'Data/glove_kws_reserved.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [9]:
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,Admission Date : 2119 - 5 - 4 Discharge Date :...,noteevents,1.000000,1.000000
1,18407,28.016667,Text,Admission Date : 2112 - 12 - 8 Discharge Date ...,noteevents,1.000000,1.000000
2,40300,155.166667,Text,Admission Date : 2194 - 7 - 18 Discharge Date ...,noteevents,1.000000,1.000000
3,23747,52.383333,Text,Admission Date : 2194 - 1 - 7 Discharge Date :...,noteevents,1.000000,1.000000
4,2357,73.133333,Text,Admission Date : 2186 - 6 - 7 Discharge Date :...,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [10]:
# data.loc[data['variable'] == 'Text', 'value'] = 1
# data

In [11]:
pred_window = 2 # hours
obs_windows = range(20, 124, 4)
# Remove test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# # Fix age.
# data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4
# # data[data.variable=='Age'][data.value>200]['value'] = 91.4

In [12]:
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
# fore_times_ip = []
# fore_values_ip = []
# fore_varis_ip = []
fore_texts_ip = []
# fore_op = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(fore_max_len-len(x))
for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)    
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    # fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    # fore_times_ip.append(np.array(list(obs_data.hour)))
    # fore_values_ip.append(np.array(list(obs_data.value)))
    
    values = list(obs_data.value)
    texts = []
    for value in values:
        texts.append(np.array([item for item in value if isinstance(item, str)]))
    fore_texts_ip.append(np.array(texts))
    
    # fore_varis_ip.append(np.array(list(obs_data.vind)))

91626it [00:00, 799862.01it/s]
  0%|          | 0/26 [00:00<?, ?it/s]/scratch/slurm_tmpdir/job_22884292/ipykernel_280184/1663465024.py:69: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(texts))
  4%|▍         | 1/26 [00:17<07:25, 17.82s/it]/scratch/slurm_tmpdir/job_22884292/ipykernel_280184/1663465024.py:69: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  fore_texts_ip.append(np.array(texts))
  8%|▊         | 2/26 [00:36<07:18, 18.27s/it]/scratch/slurm_tmpdir/job_22884292/ipykernel_280184/1663465024.py:69: VisibleDeprecatio

In [13]:
del data
# fore_times_ip = np.concatenate(fore_times_ip, axis=0)
# fore_values_ip = np.concatenate(fore_values_ip, axis=0)
# fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)
fore_texts_ip = np.concatenate(fore_texts_ip, axis=0)
# fore_op = np.concatenate(fore_op, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()

fore_train_texts_ip = [ip[train_ind] for ip in [fore_texts_ip]]
fore_valid_texts_ip = [ip[valid_ind] for ip in [fore_texts_ip]]

fore_train_texts = fore_train_texts_ip[0]
fore_valid_texts = fore_valid_texts_ip[0]

del fore_train_texts_ip
del fore_valid_texts_ip

In [14]:
# train_texts = []
# for text in tqdm(fore_train_texts):
#     train_texts.append(' '.join(text))

# train_texts = np.array(train_texts)

# del fore_train_texts

In [15]:
# np.savetxt('forecasting_train_ip.txt', train_texts, fmt='%s')

In [16]:
# gc.collect()

In [17]:
valid_texts = []
for text in tqdm(fore_valid_texts):
    valid_texts.append(' '.join(text))

valid_texts = np.array(valid_texts)

del fore_valid_texts
np.savetxt('forecasting_valid_ip.txt', valid_texts, fmt='%s')

100%|██████████| 136823/136823 [00:06<00:00, 21812.42it/s]


In [18]:
# train_val_texts = train_texts + val_texts

In [25]:
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(valid_texts)
vocab_size = len(t.word_index) + 1
vocab_size

141579

In [28]:
# encoded_docs = t.texts_to_sequences(valid_texts)
# encoded_docs

In [29]:
max_len = 0

for doc in tqdm(encoded_docs):
    if len(doc)>max_len:
        max_len = len(doc)

max_len

100%|██████████| 136823/136823 [00:00<00:00, 2427835.79it/s]


22867

In [32]:
del valid_texts

In [37]:
test_texts = np.loadtxt('forecasting_valid_ip.txt', dtype='str')

ValueError: Wrong number of columns at line 2

In [22]:
# from numpy import array, asarray, zeros
# # # GloVe
# # glove_path = 'resources/glove.840B.300d.txt'

# # # load the whole embedding into memory
# # embeddings_index = dict()
# # f = open(glove_path)
# # for line in f:
# #     values = line.split()
# #     word = values[0]
# #     coefs = asarray(values[1:], dtype='float32')
# #     embeddings_index[word] = coefs
# # f.close()
# # print('Loaded %s word vectors.' % len(embeddings_index))


# embedding_model = {}
# f = open('resources/glove.840B.300d.txt', 'r', encoding="utf8")
# for line in f:
#     values = line.split()
#     word = ''.join(values[:-300])
#     coefs = np.asarray(values[-300:], dtype='float32')
#     embedding_model[word] = coefs
# f.close()

In [23]:
# # create a weight matrix for words in training docs
# embedding_matrix = zeros((vocab_size, 300))
# for word, i in t.word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [24]:
# embedding_matrix.shape